In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
def stock_price(S0,T): 
    #parameters are  given in the stock dynamics equation
    N=252
    mu=0.05
    sigma=0.30
    Z=[]
    t = np.linspace(0, T, N+1)
    dt = float(T/N)
    Z.append(S0)
    for i in np.arange(0,N):
        drift = mu*(t[i+1]-t[i])*Z[i]
        diffusion = sigma*np.random.normal(0,1)*math.sqrt(t[i+1]-t[i])*Z[i]
        S_change=drift + diffusion
        Z.append(Z[-1]+S_change)
    return Z


In [3]:
a=np.zeros((252,10000))
S0=40
a=[np.hstack(stock_price(40,1.0)) for i in np.arange(0,a.shape[1])]
X=np.array(a)
X.reshape(253,10000)
X=pd.DataFrame(a).T
X

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
1,39.674567,40.427288,39.848027,38.376387,40.421107,40.184542,39.622253,42.002761,40.019713,40.871990,...,40.206768,40.820563,40.099784,40.744940,41.289546,40.964138,40.319698,40.377499,39.929342,39.719643
2,38.281343,40.024386,39.912539,38.291120,41.947158,40.065332,39.842546,40.863085,40.752058,41.187742,...,41.007185,41.494914,40.863649,41.419030,39.970531,41.338947,39.804409,39.828045,39.151122,38.864325
3,38.663490,40.794816,38.395137,38.072201,42.472151,39.902791,40.892525,42.858503,41.069993,42.097175,...,41.263639,41.430526,40.474322,41.615472,40.366326,40.164412,40.090439,39.440635,39.719980,39.804858
4,38.242673,41.249971,38.574028,37.063969,42.132397,40.470200,41.711337,43.803363,41.486796,41.388673,...,40.291846,40.922160,40.938106,41.827246,39.191029,40.341555,40.502588,40.066989,39.738244,40.702036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,61.297117,50.043450,33.327886,52.596714,52.748673,41.031601,33.447521,31.171671,32.574556,29.564546,...,31.091359,67.007911,30.215726,26.730080,36.545193,61.764367,35.555530,40.035610,27.558028,68.139928
249,60.177020,50.977451,33.208977,51.640178,51.530385,40.785628,33.912894,31.504192,32.534554,29.681977,...,30.760705,68.036608,28.551840,26.308101,36.809127,61.854598,35.154480,41.067561,27.928380,67.055772
250,60.565235,50.378572,32.283349,51.314602,53.519377,41.845483,33.730255,32.154815,32.655268,29.421192,...,31.611689,66.702974,29.276045,27.099333,37.344228,62.517932,35.338717,40.376647,27.429495,67.836055
251,61.703633,49.870726,31.706006,49.284588,54.749332,43.983912,34.649400,31.663877,32.754592,29.189940,...,31.299853,65.842048,29.644402,26.418951,38.482934,62.578886,35.139144,40.076947,28.034911,66.320177


In [4]:
barrier=35
Mat=1.0
r0=0.05
Strike=40

payoff=np.zeros(X.shape[1])
for i in range(0,X.shape[1]):
    if min(X.loc[:,i]) < barrier:
        payoff[i] = 0.0
    else:
        payoff[i] = max(Strike-X.iloc[-1,i],0)

option_price=np.mean(payoff)*math.exp(-r0 * Mat)
print('The price of the knockc out ''(Down and out)'' put is {:.2f}'.format(option_price))
variance_of_payoff=np.std(payoff)
lower_bound=np.round(option_price - 1.96 *variance_of_payoff/math.sqrt(X.shape[1]),2 )
upper_bound=np.round(option_price + 1.96 *variance_of_payoff/math.sqrt(X.shape[1]),2 )
print('The 95% confidence interval for the option price is {:.2f} and {:.2f}'.format(lower_bound,upper_bound))


The price of the knockc out (Down and out) put is 0.07
The 95% confidence interval for the option price is 0.06 and 0.08


In [5]:
payoff=np.zeros(X.shape[1])
for i in range(0,X.shape[1]):
    if min(X.loc[:,i]) >= barrier:
        payoff[i] = 0.0
    else:
        payoff[i] = max(Strike-X.iloc[-1,i],0)

option_price1=np.mean(payoff)*math.exp(-0.05 * 1.0)
print('The price of the knock in put is {:.2f}'.format(option_price1))
lower_bound=np.round(option_price1 - 1.96 *variance_of_payoff/math.sqrt(X.shape[1]),2 )
upper_bound=np.round(option_price1 + 1.96 *variance_of_payoff/math.sqrt(X.shape[1]),2 )
print('The 95% confidence interval for the option price is {:.2f} and {:.2f}'.format(lower_bound,upper_bound))


The price of the knock in put is 3.71
The 95% confidence interval for the option price is 3.70 and 3.71


In [6]:
from BS_Option_Price_Impl import blackscholes_price
price=blackscholes_price(40,40,1.0,0.05,0.30, option='put')
print('The black-scholes put option  price is {:.2f}'.format(price))

The black-scholes put option  price is 3.74
